# 🔬 AI-Driven OLED Assistant (Domain-Specific RAG)

## 🎯 Project Overview

**Goal:** Build an intelligent AI assistant for OLED display engineering and technical support.

**Key Features:**
- 🛡️ Strict RAG: Answers based on documents, and rejects if relevant documents are not available
- 🎯 Relevance Score: Similarity score + sigmoid transformation-based 3-tier decision system (RAG / NO_ANSWER / OFF_TOPIC)
- 📈 Quantitative Evaluation: Quantitative evaluation of Strict RAG answer quality using LLM-as-a-judge (Specificity / Relevance / Factuality)
- 💼 Engineering Support: Answers questions about OLED processes, device properties, and optical simulation

---
## 🛠️ Required Packages

### 📦 Core Dependencies
- **langchain**: LangChain framework (RAG pipeline construction)
- **langchain-community**: PDF/DOCX loaders and integration features
- **langchain-openai**: Chat API wrapper for OpenAI and OpenAI-compatible servers (e.g., Ollama/vLLM)
- **docarray**: Python vector search engine (lightweight and fast)
- **pypdf**: PDF text extraction
- **docx2txt**: DOCX text extraction
- **tiktoken**: Token calculation (for cost estimation)
- **python-dotenv**: Environment variable management (.env)

In [ ]:
# Cell 1
# %pip install --upgrade pip
# %pip install -U langchain langchain-community langchain-openai
# %pip install -U docarray pypdf docx2txt tiktoken openai python-dotenv
# %pip install -U sentence-transformers  # 🔑 For HuggingFace embeddings (sentence-transformers/all-MiniLM-L6-v2)

## 🔐 Environment Variables and API Key Configuration

**What This Section Does:**
- Loads environment variables from the `.env` file in the project directory
- By default looks for `OPENAI_API_KEY`, but **this notebook's Strict RAG experiment uses local Mistral-Nemo (ollama_local)**, so it's not required
- Loads the key in advance for future evaluation with OpenAI-based LLMs (e.g., gpt-4o-mini)
- The `python-dotenv` package automatically reads the `.env` file

In [ ]:
# Cell 3: Load OpenAI API Key from .env file
# This cell loads environment variables (especially the API key) before any API calls

import os
from dotenv import load_dotenv

# Load all variables from .env file into environment
# The .env file should be in the same directory as this notebook
load_dotenv()

# Check if API key was successfully loaded
if os.environ.get("OPENAI_API_KEY"):
    print("✅ OpenAI API key loaded successfully from .env file")
else:
    print("❌ Warning: OPENAI_API_KEY not found in .env file")
    print("   Please create a .env file with: OPENAI_API_KEY=your-key-here")

## ⚙️ Configuration (Hyperparameters)

These are the main hyperparameters you can adjust to customize the OLED Assistant system.

In [ ]:
# Cell 5: User Configuration (Hyperparameters)
# =======================================
# 🔬 Hyperparameter Tuning Strategy (Strict RAG):
# =======================================
# Step 1: RAG Answer Quality Optimization
#   - Tuning parameters: CHUNK_SIZE, CHUNK_OVERLAP, TOP_K_DOCUMENTS
#
# Step 2: Strict RAG Threshold Settings
#   - Tuning parameters: RELEVANCE_THRESHOLD
#   - Goal: Boldly reject if content is not in documents

DOCS_FOLDER = "../data"  # Folder where OLED technical documents are stored
DB_PATH = "../chroma_db" # Vector DB storage path (Persistence)
# 🔁 LLM Configuration: Currently using local Mistral-Nemo 12B (Ollama)
LLM_MODEL = "mistral-nemo"      # Must match ollama model name
LLM_PROVIDER = "llama_local"    # OpenAI → Local OpenAI-compatible server
# ========================================
# 🔧 STEP 1: RAG Answer Quality Settings
# ========================================
# 📐 CHUNK_SIZE: Chunk size (character count)
#    - Smaller: Precise search, but risk of context fragmentation
#    - Larger: Context preservation, but risk of topic mixing
CHUNK_SIZE = 3000  

# 📐 CHUNK_OVERLAP: Overlap between chunks (character count)
#    - Ensures context connection, prevents information loss at boundaries
CHUNK_OVERLAP = 500  

# 📐 TOP_K_DOCUMENTS: Number of chunks to retrieve
#    - Smaller: Faster, only accurate top results
#    - Larger: Diverse perspectives, but noise increases
TOP_K_DOCUMENTS = 4 

# 📐 LLM_TEMPERATURE: Answer creativity
#    - 0.0: Deterministic, fact-based (suitable for Strict RAG)
#    - 0.7+: Creative, diverse expressions
LLM_TEMPERATURE = 0.2  

# ========================================
# ⚙️ STEP 2: Strict RAG Settings
# ========================================
# 📐 SIGMOID_MIDPOINT: Center point of score distribution
#    - Higher: Lower scores become even lower (stricter)
#    - Needs adjustment after testing with new embedding model
SIGMOID_MIDPOINT = 0.50  

# 📐 SIGMOID_STEEPNESS: Score separation intensity
#    - Higher: Middle scores pushed to extremes
SIGMOID_STEEPNESS = 18  

# 📐 RELEVANCE_THRESHOLD: Answerability threshold
#    - Returns 'No Answer' if below this score
#    - Needs adjustment after testing with new embedding model
RELEVANCE_THRESHOLD = 0.6

# 6️⃣ Test Question List
# Various questions to test Relevance Score distribution
# Expected distribution: High(0.8+) → Medium(0.5-0.7) → Low(0.2-0.4) → Very Low(<0.2)
TEST_QUERIES = [
    # Case 1: High relevance, must-be-RAG
    "What are the key degradation mechanisms of a blue phosphorescent OLED?",
    
    # Case 2: High but slightly lower than Case 1
    "How does exciton diffusion length affect charge separation efficiency in organic semiconductor devices?",
    
    # Case 3: Display but non-OLED
    "What are the major challenges in mass transfer processes for MicroLED displays?",  
    
    # Case 4: Non-display semiconductor device
    "How does doping concentration affect the electron mobility in silicon MOSFETs?",
    
    # Case 5: Science but non-electronics
    "Describe the physical principles used to reduce aerodynamic drag in automotive design",

    # Case 6: Non-science, off-topic
    "Recommend a good hiking trail near Santa Clara for a weekend trip",  
]

print("=" * 80)
print("✅ Configuration Complete!")
print("=" * 80)
print(f"📂 Documents Folder: {DOCS_FOLDER}")
print(f"💾 DB Path: {DB_PATH}")
print("=" * 80)

## 📊 Logging, Monitoring & Error Handling Setup

**Why add this before hyperparameter tuning?**

When experimenting with various hyperparameters, the following are needed:
1. 📝 **Logging**: Record what's happening (for debugging)
2. ⏱️ **Monitoring**: Track performance (response time, cost)
3. 🛡️ **Error Handling**: Handle errors gracefully (API failures, timeouts)

---

### 🎯 What We Will Track

**Performance Metrics:**
- ⏱️ Response time per query
- 💰 Token usage and estimated cost
- 📊 Mode distribution (RAG vs NO_ANSWER vs OFF_TOPIC)

**Error Handling:**
- ⚠️ LLM API errors (Rate limit, timeout from OpenAI or local Mistral and other OpenAI-compatible servers)
- 📁 Document loading errors (file not found, corrupted PDF)
- 🔄 Automatic retry on transient errors

**Experiment Tracking:**
- 💾 Automatically save results to CSV file
- 📈 Compare various hyperparameter settings
- 🎯 Find optimal settings based on data

---

In [ ]:
# Cell 7: Logging, Monitoring & Error Handling Setup
# This cell sets up logging, cost tracking, error handling, and experiment tracking

import logging
import time
import csv
import os
from datetime import datetime
from functools import wraps
import tiktoken

# ========================================
# 📝 LOGGING SETUP
# ========================================
# Create logs directory if it doesn't exist
os.makedirs("logs", exist_ok=True)

# Configure logging
# Note: remove existing handlers before reconfiguring so the cell can be run multiple times safely
# Remove all existing handlers so repeated cell runs behave reliably
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
    handler.close()  # Clean up handler resources

# Then apply the base configuration
log_filename = f'logs/AI_OLED_assistant_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_filename),
        logging.StreamHandler()  # Also print to console
    ],
    force=True  # Python 3.8+: force reconfiguration to avoid duplicate handlers
)

logger = logging.getLogger(__name__)
logger.info(f"📝 Logging initialized: {log_filename}")

# ========================================
# 💰 COST TRACKING
# ========================================
class CostTracker:
    """
    Track token usage and estimated API costs.
    
    OpenAI pricing (as of 2024):
    - GPT-4o-mini: $0.150 per 1M input tokens, $0.600 per 1M output tokens
    """
    
    # Pricing per 1M tokens (USD)
    PRICING = {
        "gpt-5": {"input": 1.25, "output": 10.00},
        "gpt-5-mini": {"input": 0.25, "output": 2.00},
        "gpt-4o-mini": {"input": 0.15, "output": 0.60},
        # Local LLMs (Mistral, etc.) have cost set to 0, only monitor tokens
        "mistral-nemo": {"input": 0.0, "output": 0.0},
    }
    
    def __init__(self, model_name="gpt-4o-mini"):
        """Initialize cost tracker for a specific model."""
        self.model_name = model_name
        # Use safe default encoding in case tiktoken doesn't know the model name
        try:
            self.encoding = tiktoken.encoding_for_model(model_name)
        except Exception:
            self.encoding = tiktoken.get_encoding("cl100k_base")
        self.total_input_tokens = 0
        self.total_output_tokens = 0
    
    def count_tokens(self, text):
        """Count tokens in a text string."""
        return len(self.encoding.encode(text))
    
    def add_tokens(self, input_text, output_text):
        """
        Add token counts for input and output.
        
        Args:
            input_text: Input prompt text
            output_text: Model response text
        """
        input_tokens = self.count_tokens(input_text)
        output_tokens = self.count_tokens(output_text)
        
        self.total_input_tokens += input_tokens
        self.total_output_tokens += output_tokens
        
        return input_tokens, output_tokens
    
    def get_cost(self):
        """Calculate total cost in USD."""
        if self.model_name not in self.PRICING:
            return 0.0  # Unknown model
        
        pricing = self.PRICING[self.model_name]
        input_cost = (self.total_input_tokens / 1_000_000) * pricing["input"]
        output_cost = (self.total_output_tokens / 1_000_000) * pricing["output"]
        
        return input_cost + output_cost
    
    def get_summary(self):
        """Get summary of token usage and cost."""
        return {
            "input_tokens": self.total_input_tokens,
            "output_tokens": self.total_output_tokens,
            "total_tokens": self.total_input_tokens + self.total_output_tokens,
            "estimated_cost_usd": self.get_cost()
        }
    
    def reset(self):
        """Reset token counters."""
        self.total_input_tokens = 0
        self.total_output_tokens = 0

# Initialize global cost tracker
cost_tracker = CostTracker(model_name=LLM_MODEL)

# ========================================
# 🛡️ ERROR HANDLING UTILITIES
# ========================================
def retry_on_api_error(max_retries=2, delay=2):
    """
    Decorator to retry function on API errors.
    
    Args:
        max_retries: Maximum number of retry attempts
        delay: Delay in seconds between retries
    
    Usage:
        @retry_on_api_error(max_retries=2, delay=2)
        def api_call():
            # Your API call here
            pass
    """
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            
            for attempt in range(max_retries + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    last_exception = e
                    error_type = type(e).__name__
                    
                    # Log the error
                    if attempt < max_retries:
                        logger.warning(f"⚠️ {error_type} on attempt {attempt + 1}/{max_retries + 1}: {str(e)}")
                        logger.info(f"🔄 Retrying in {delay} seconds...")
                        time.sleep(delay)
                    else:
                        logger.error(f"❌ Failed after {max_retries + 1} attempts: {str(e)}")
            
            # If all retries failed, raise the last exception
            raise last_exception
        
        return wrapper
    return decorator

def safe_file_load(file_path, loader_class):
    """
    Safely load a file with error handling.
    
    Args:
        file_path: Path to file
        loader_class: LangChain loader class (PyPDFLoader or Docx2txtLoader)
    
    Returns:
        List of loaded documents, or empty list if failed
    """
    try:
        loader = loader_class(file_path)
        documents = loader.load()
        logger.info(f"✅ Loaded: {os.path.basename(file_path)} ({len(documents)} pages/sections)")
        return documents
    except FileNotFoundError:
        logger.error(f"❌ File not found: {file_path}")
        return []
    except PermissionError:
        logger.error(f"❌ Permission denied: {file_path}")
        return []
    except Exception as e:
        logger.error(f"❌ Error loading {os.path.basename(file_path)}: {type(e).__name__} - {str(e)}")
        return []

# ========================================
# 📊 EXPERIMENT TRACKING
# ========================================
class ExperimentTracker:
    """Track and save hyperparameter experiment results."""
    
    def __init__(self, csv_filename="hyperparameter_experiments.csv"):
        """Initialize experiment tracker."""
        self.csv_filename = csv_filename
        self.current_experiment = {}
        
        # Create CSV file with headers if it doesn't exist
        if not os.path.exists(csv_filename):
            with open(csv_filename, 'w', newline='') as f:
                # CSV header for Strict RAG (no LLM baseline comparison)
                writer = csv.DictWriter(f, fieldnames=[
                    'timestamp',  # 1

                    # RAG Quality (Step 1)
                    'chunk_size', 'chunk_overlap', 'top_k', 'temperature',  # 2-5

                    # System Architecture (Step 2) - Only what's used in Strict RAG
                    'relevance_threshold', 'sigmoid_midpoint', 'sigmoid_steepness',  # 6-8

                    # Performance
                    'avg_response_time_sec', 'total_tokens', 'estimated_cost_usd',  # 9-11

                    # Mode Distribution (Strict RAG: RAG, NO_ANSWER_IN_DOCS, OFF_TOPIC only)
                    'mode_rag_count', 'mode_no_answer_count', 'mode_off_topic_count', 'mode_error_count',  # 12-15

                    # Score Metrics (RAG only, no LLM baseline)
                    'avg_relevance_score', 'avg_rag_score',  # 16-17
                    'avg_rag_specificity', 'avg_rag_relevance', 'avg_rag_factuality',  # 18-20

                    # Relevance separation metrics (Q1–Q2 vs Q5–Q6)
                    'rel_mean_pos', 'rel_mean_neg', 'rel_gap', 'rel_margin',  # 21-24

                    'notes'  # 25
                ])
                writer.writeheader()
    
    def start_experiment(self, config):
        """
        Start tracking a new experiment.
        
        Args:
            config: Dictionary with hyperparameter settings
        """
        # Strict RAG modes: RAG, NO_ANSWER_IN_DOCS, OFF_TOPIC only
        self.current_experiment = {
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            **config,
            'response_times': [],
            'mode_counts': {
                'RAG': 0, 
                'NO_ANSWER_IN_DOCS': 0,
                'OFF_TOPIC': 0,
                'ERROR': 0
            }
        }
        logger.info(f"🧪 Starting experiment with config: {config}")
    
    def log_query(self, response_time, mode):
        """Log a single query result."""
        self.current_experiment['response_times'].append(response_time)
        self.current_experiment['mode_counts'][mode] += 1
    
    def save_experiment(self, 
                       avg_relevance_score=None, avg_rag_score=None,
                       avg_rag_specificity=None, avg_rag_relevance=None, avg_rag_factuality=None,
                       rel_mean_pos=None, rel_mean_neg=None, rel_gap=None, rel_margin=None,
                       notes=""):
        """Save Strict RAG experiment results to CSV.

        Args
        ----
        avg_relevance_score : float | None
            Average relevance score (0-1)
        avg_rag_score : float | None
            Average RAG answer quality (1-10)
        avg_rag_specificity : float | None
            Average RAG specificity score (1-10)
        avg_rag_relevance : float | None
            Average RAG relevance score (1-10)
        avg_rag_factuality : float | None
            Average RAG factuality score (1-10)
        rel_mean_pos : float | None
            Q1–Q2 (OLED core) relevance average
        rel_mean_neg : float | None
            Q5–Q6 (OFF-TOPIC) relevance average
        rel_gap : float | None
            rel_mean_pos - rel_mean_neg
        rel_margin : float | None
            min(Q1–Q2) - max(Q5–Q6)
        notes : str
            Additional notes about the experiment
        """
        cost_summary = cost_tracker.get_summary()
        
        # Calculate averages
        avg_response_time = sum(self.current_experiment['response_times']) / len(self.current_experiment['response_times']) if self.current_experiment['response_times'] else 0
        
        # Row data for Strict RAG (matches CSV header order)
        row = {
            'timestamp': self.current_experiment['timestamp'],

            # RAG Quality (Step 1)
            'chunk_size': self.current_experiment.get('chunk_size'),
            'chunk_overlap': self.current_experiment.get('chunk_overlap'),
            'top_k': self.current_experiment.get('top_k'),
            'temperature': self.current_experiment.get('temperature'),

            # System Architecture (Step 2) - Only what's used in Strict RAG
            'relevance_threshold': self.current_experiment.get('relevance_threshold'),
            'sigmoid_midpoint': self.current_experiment.get('sigmoid_midpoint'),
            'sigmoid_steepness': self.current_experiment.get('sigmoid_steepness'),
            
            # Performance
            'avg_response_time_sec': f"{avg_response_time:.2f}",
            'total_tokens': cost_summary['total_tokens'],
            'estimated_cost_usd': f"${cost_summary['estimated_cost_usd']:.4f}",

            # Mode Distribution (Strict RAG: RAG, NO_ANSWER_IN_DOCS, OFF_TOPIC only)
            'mode_rag_count': self.current_experiment['mode_counts']['RAG'],
            'mode_no_answer_count': self.current_experiment['mode_counts']['NO_ANSWER_IN_DOCS'],
            'mode_off_topic_count': self.current_experiment['mode_counts']['OFF_TOPIC'],
            'mode_error_count': self.current_experiment['mode_counts']['ERROR'],
            
            # Score Metrics (RAG only, no LLM baseline)
            'avg_relevance_score': f"{avg_relevance_score:.3f}" if avg_relevance_score is not None else "N/A",
            'avg_rag_score': f"{avg_rag_score:.2f}" if avg_rag_score is not None else "N/A",
            'avg_rag_specificity': f"{avg_rag_specificity:.2f}" if avg_rag_specificity is not None else "N/A",
            'avg_rag_relevance': f"{avg_rag_relevance:.2f}" if avg_rag_relevance is not None else "N/A",
            'avg_rag_factuality': f"{avg_rag_factuality:.2f}" if avg_rag_factuality is not None else "N/A",

            # Relevance separation metrics (Q1–Q2 vs Q5–Q6)
            'rel_mean_pos': f"{rel_mean_pos:.3f}" if rel_mean_pos is not None else "N/A",
            'rel_mean_neg': f"{rel_mean_neg:.3f}" if rel_mean_neg is not None else "N/A",
            'rel_gap': f"{rel_gap:.3f}" if rel_gap is not None else "N/A",
            'rel_margin': f"{rel_margin:.3f}" if rel_margin is not None else "N/A",

            'notes': notes
        }
        
        # Append to CSV
        with open(self.csv_filename, 'a', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=row.keys())
            writer.writerow(row)
        
        logger.info(f"💾 Experiment saved to {self.csv_filename}")
        logger.info(f"📊 Summary: Avg time={avg_response_time:.2f}s, Cost=${cost_summary['estimated_cost_usd']:.4f}")

# Initialize global experiment tracker
experiment_tracker = ExperimentTracker()

print("✅ Logging, Monitoring & Error Handling configured successfully!")
print(f"📝 Logs will be saved to: logs/")
print(f"📊 Experiment results will be saved to: hyperparameter_experiments.csv")
print(f"💰 Cost tracking enabled for model: {LLM_MODEL}")

## 📖 How to Use Logging, Monitoring & Error Handling

**✅ Automatic Logging:**
- All operations are logged to `logs/AI_OLED_assistant_[timestamp].log`.
- Logs include timestamps, error messages, and performance metrics.
- Monitoring runs automatically in the background.

**✅ Cost Tracking:**
- Token usage for each query is automatically calculated.
- Estimated API cost is calculated in real time.
- A summary is printed after each experiment.

**✅ Error Handling:**
- API failures automatically retry (up to 2 times).
- File loading errors are caught and written to the logs.
- User-friendly error messages are returned.

**✅ Experiment Tracking:**
- All hyperparameter experiments are saved to `hyperparameter_experiments.csv`.
- Includes: response time, token usage, cost, mode distribution, and more.

---

## 📁 Document Loading Setup

**What This Section Does:**
This is a configuration section for document loading. The actual loading happens in the next section.

**Environment:**
- Designed for local execution
- Reads OLED technical documents directly from the `data/` folder

## 📄 Document Loading and Chunking

**What This Section Does:**
This is where OLED technical documents are actually loaded and processed.

**Step-by-step Process:**

1. **File Discovery** (`glob.glob` with `recursive=True`)
   - Scans for PDF/DOCX files in the `data/` folder and **all subfolders**
   - Automatically recognizes documents even if organized in folders

2. **Document Loading** (`PyPDFLoader`, `Docx2txtLoader`)
   - PDF → text conversion (one page = one document)
   - DOCX → text conversion (one section = one document)
   - `safe_file_load()` wrapper protects against crashes from corrupted files

3. **Text Chunking** (`RecursiveCharacterTextSplitter`)
   - Splits long documents into smaller chunks
   - Each chunk is `CHUNK_SIZE` characters (e.g., current setting: 3000)
   - `CHUNK_OVERLAP` characters of overlap between chunks (e.g., current setting: 500)
   - Why overlap? Prevents context loss when sentences are split at chunk boundaries

**Why We Chunk Documents:**
- **LLM Token Limits**: Models have a maximum input size (e.g., 128K tokens)
- **Better Retrieval**: Smaller chunks match queries more accurately
- **Context Preservation**: Overlap prevents important information loss at boundaries

In [ ]:
# Cell 10: Load and Chunk Documents
# This cell loads all PDF and DOCX files from the docs folder and splits them into chunks

import os
import glob
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ========================================
# 0. Fast path: If ChromaDB already exists, skip loading & chunking
#    - If DB exists, skip PDF loading + chunking, and only load existing DB in Cell 12
# ========================================
if os.path.exists(DB_PATH) and os.listdir(DB_PATH):
    print(f"📂 Existing ChromaDB found at {DB_PATH}")
    print("⏭️ Skipping document loading & chunking (using existing DB only).")
    logger.info(f"Skip loading/chunking because DB already exists at {DB_PATH}")

else:
    # ========================================
    # STEP 1: Find all PDF and DOCX files
    # ========================================
    logger.info(f"📂 Scanning folder: {DOCS_FOLDER}")

    # Check if docs folder exists before proceeding
    if not os.path.exists(DOCS_FOLDER):
        logger.error(f"❌ Docs folder not found: {DOCS_FOLDER}")
        raise FileNotFoundError(f"Documents folder '{DOCS_FOLDER}' does not exist")

    # Use glob to find all PDF and DOCX files in the folder (including subfolders)
    # Use recursive=True and ** to search subfolders as well
    pdf_files = glob.glob(os.path.join(DOCS_FOLDER, "**/*.pdf"), recursive=True)
    docx_files = glob.glob(os.path.join(DOCS_FOLDER, "**/*.docx"), recursive=True)

    print(f"📂 Found {len(pdf_files)} PDF files and {len(docx_files)} DOCX files in {DOCS_FOLDER}/ (including subfolders)")
    logger.info(f"Found {len(pdf_files)} PDFs and {len(docx_files)} DOCX files")

    # Check if any files found - warn if folder is empty
    if len(pdf_files) == 0 and len(docx_files) == 0:
        logger.warning(f"⚠️ No PDF or DOCX files found in {DOCS_FOLDER}/")
        print(f"⚠️ WARNING: No documents found. Please add PDF or DOCX files to {DOCS_FOLDER}/")

    # ========================================
    # STEP 2: Load all documents into memory
    # ========================================
    all_documents = []  # This will store all loaded document pages/sections

    # Load PDF files using PyPDFLoader (one page = one document)
    # safe_file_load() wraps the loader with error handling (won't crash on corrupt files)
    print(f"\n📄 Loading PDF files...")
    for pdf_file in pdf_files:
        # safe_file_load() handles errors gracefully - returns [] on failure
        documents = safe_file_load(pdf_file, PyPDFLoader)
        all_documents.extend(documents)  # Add all pages from this PDF to our collection

    # Load DOCX files using Docx2txtLoader (one section = one document)
    print(f"\n📄 Loading DOCX files...")
    for docx_file in docx_files:
        documents = safe_file_load(docx_file, Docx2txtLoader)
        all_documents.extend(documents)

    # ========================================
    # STEP 3: Validate that documents were loaded
    # ========================================
    if len(all_documents) == 0:
        logger.error("❌ No documents were successfully loaded")
        raise ValueError("Failed to load any documents. Please check file formats and permissions.")

    print(f"\n✅ Total documents loaded: {len(all_documents)}")
    logger.info(f"Successfully loaded {len(all_documents)} document sections")

    # ========================================
    # STEP 4: Split documents into chunks
    # ========================================
    # RecursiveCharacterTextSplitter splits text intelligently:
    # - Tries to split at paragraph breaks first, then sentences, then words
    # - Ensures chunks are approximately CHUNK_SIZE characters
    # - Adds CHUNK_OVERLAP characters of overlap between chunks to preserve context
    try:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=CHUNK_SIZE,      # Target size: CHUNK_SIZE characters per chunk
            chunk_overlap=CHUNK_OVERLAP  # Overlap: CHUNK_OVERLAP characters between chunks
        )
        # Split all documents into chunks
        docs = text_splitter.split_documents(all_documents)
        # What split_documents() does:
            # 1. Reads each Document's page_content.
            # 2. Splits text into ~CHUNK_SIZE-character chunks (paragraph/sentence boundaries preferred).
            # 3. Creates a new Document object for each chunk.
            # 4. Preserves original metadata (source, page, etc.).
        # Result: docs = [chunk_1, chunk_2, chunk_3, ...]
        # Each chunk is a Document object
        print(f"🔹 Total chunks created: {len(docs)}")
        logger.info(f"Created {len(docs)} text chunks (size={CHUNK_SIZE}, overlap={CHUNK_OVERLAP})")
    except Exception as e:
        logger.error(f"❌ Error splitting documents: {str(e)}")
        raise

## 🧠 Embedding and Vector Store Creation

**What This Section Does:**
Converts all document chunks into numerical vectors (embeddings) and builds a searchable index.

**Why We Need Embeddings:**
- Raw text cannot be directly compared by computers
- Embeddings convert text into numerical vectors that capture **semantic meaning**
- Similar texts → similar vectors → can search for relevant documents by comparing vectors

**Process:**

1. **Initialize Embedding Model**
   - 🔥 Uses **BAAI/bge-m3** (multilingual·multitask embedding, medium size)
   - Based on sentence-transformers, so can be used directly with `HuggingFaceEmbeddings`
   - Why? Lighter than gte-large, runs at realistic speeds on local Mac (MPS) environments while maintaining excellent search quality

2. **Generate Vector Embeddings**
   - Each document chunk is converted into a vector (high-dimensional numeric array)
   - These vectors represent the **semantic meaning** of the text

3. **Build Vector Store** (`ChromaDB`)
   - Stores all embeddings for fast search
   - Builds an index that enables similarity search
   - Supports local persistent storage

**Next Steps:**
- When you ask a question, the query is also converted into an embedding
- The system compares the query embedding against all document embeddings
- Returns the most similar documents (based on cosine similarity score)

**Key Advantages:**
- Goes beyond simple keyword matching (understands that "OLED" and "organic light-emitting diode" are related)
- Captures semantic relationships (e.g., "phosphorescent material" vs "blue phosphorescence")

In [ ]:
# Cell 12: Vector Store Construction (ChromaDB with Persistence)
# This cell embeds documents and saves them to ChromaDB, or loads existing DB if it exists.
import os
import time
import shutil
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# ========================================
# STEP 1: Initialize Embedding Model
# ========================================
# 🔥 BAAI/bge-m3: Medium-sized embedding model for multilingual and multi-task use
#    - sentence-transformers family → Easy to use with HuggingFaceEmbeddings
#    - Lighter than gte-large and realistic speed on Mac (MPS)
EMBEDDING_MODEL = "BAAI/bge-m3"

try:
    print("🔄 Loading embedding model...")
    print(f"📦 Model: {EMBEDDING_MODEL}")
    
    import torch
    
    # Device configuration
    if torch.cuda.is_available():
        device = 'cuda'
    elif torch.backends.mps.is_available():
        device = 'mps'
    else:
        device = 'cpu'
    print(f"✅ Using device: {device.upper()}")
    
    # Load BGE-m3 embedding model
    embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL,
        model_kwargs={
            'device': device,
        },
        encode_kwargs={
            'normalize_embeddings': True,  # Normalize for cosine similarity
            'batch_size': 16,              # 🔑 Batch size (adjustable between 8~32)
        }
    )
    print(f"✅ Embedding model loaded! ({EMBEDDING_MODEL})")
    
except Exception as e:
    print(f"❌ Failed to load embedding model: {e}")
    raise

# ========================================
# STEP 2: Load or Create ChromaDB (Check -> Exist?)
# ========================================
def get_vectorstore():
    # Check if DB folder exists and is not empty
    if os.path.exists(DB_PATH) and os.listdir(DB_PATH):
        print(f"\n📂 Found existing ChromaDB at {DB_PATH}")
        print("🔄 Loading existing DB... (skipping embedding)")
        
        vectorstore = Chroma(
            persist_directory=DB_PATH,
            embedding_function=embeddings
        )
        print("✅ Existing DB loaded!")
        return vectorstore
    else:
        print(f"\n🆕 No existing DB found. Embedding documents from scratch.")
        print(f"🔄 Embedding {len(docs)} chunks... (this may take time)")
        
        # If empty folder exists, delete and recreate
        if os.path.exists(DB_PATH):
            shutil.rmtree(DB_PATH)
            
        vectorstore = Chroma.from_documents(
            documents=docs,
            embedding=embeddings,
            persist_directory=DB_PATH
        )
        # Chroma automatically calls persist(), but explicitly save
        # vectorstore.persist() # Latest version auto-saves
        print(f"✅ New DB created and saved: {DB_PATH}")
        return vectorstore

try:
    vectorstore = get_vectorstore()
    # Verify data
    count = vectorstore._collection.count()
    print(f"📊 Number of document chunks stored in DB: {count}")
except Exception as e:
    print(f"❌ Error processing vector store: {e}")
    raise

## 🎯 Strict RAG System

**What This Is:**
This is the **core engine** of OLED Assistant. It's a Strict RAG system that uses documents as the **primary source** and only allows OLED/physics knowledge as supplementary when context is partial.

**Problem It Solves:**
- If answer is in documents → Prioritize documents and answer with RAG (with limited LLM knowledge support if needed)
- If relevant content is not in documents → Clearly handle as "No Answer" or "Information not in documents"
- Questions clearly unrelated to OLED → Auto-reject (OFF_TOPIC)

**How the Decision Process Works:**

1. **Calculate Relevance Score (0.0 - 1.0)**
   - Convert query to embedding
   - Compare with document embeddings
   - **Sigmoid transformation** amplifies separation:
     - OLED-related queries → boosted to 0.80–0.99 range
     - Unrelated queries → pushed to 0.01–0.20 range
   - Average similarity after sigmoid = **Relevance Score**

2. **Three-Tier Decision System:**

   **TIER 1: RAG Mode** (Relevance ≥ `RELEVANCE_THRESHOLD`)
   - Documents are highly relevant
   - ✅ Use RAG directly (documents + LLM)
   - Result: 🟢 Green answer

   **TIER 2: No Information in Documents** (RAG returns "Information not found")
   - Documents are relevant but no answer to specific question
   - ✅ Return "Content not available"
   - Result: 🟠 Orange (information absence)

   **TIER 3: Off-Topic Auto-Reject** (Relevance < `RELEVANCE_THRESHOLD`)
   - Clearly unrelated to OLED display
   - ✅ Immediately reject (no LLM call → saves cost)
   - Result: 🔴 Red (auto-rejected)

---

### 📊 Understanding the Score System

**1. Relevance Score (0.0 – 1.0)**
- **Purpose**: "Are the documents relevant to this query?"
- **Calculation Method:**
  1. Query → embedding vector
  2. Compare with top-K document embeddings (cosine similarity)
  3. Calculate average similarity
  4. Apply sigmoid transformation (amplifies separation)
- **Usage**: Decides which mode to use

**2. RAG Quality Score (1–10)**
- **Purpose**: "How good is the RAG answer?"
- **Calculation Method:** LLM evaluates on 3 metrics and averages:
  - **Specificity** (1–10): How specific is the answer?
  - **Relevance** (1–10): Does it actually answer the question?
  - **Factuality** (1–10): Does it contain verifiable facts and data?
- **Usage**: RAG system performance evaluation

**Key Distinction:**
- **Relevance Score** = Quality of **document matching** (input quality)
- **RAG Score** = Quality of the **answer itself** (output quality)
- These are **independent** – high relevance but low quality answers can occur

In [ ]:
# Cell 14: Strict RAG Advisor Implementation
# This cell implements a Strict RAG system that answers only based on content in documents.
import json
import math
import os
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from IPython.display import HTML, display


def create_llm(provider: str, model_name: str, temperature: float):
    """LLM factory function.
    
    - provider="openai"  → OpenAI official endpoint (e.g., gpt-4o-mini)
    - provider="llama_local" → OpenAI-compatible local/on-premise endpoint (e.g., Ollama, vLLM, internal LLM server, etc.)
    
    By separating this, you can easily swap between OpenAI-based models ↔ local Mistral·Llama·Gemma etc.
    by just changing provider / model_name later.
    """
    if provider == "openai":
        # Current setting: OpenAI API (e.g., gpt-4o-mini)
        return ChatOpenAI(model=model_name, temperature=temperature)
    elif provider == "llama_local":
        # Local/on-premise OpenAI-compatible server (e.g., Ollama, vLLM)
        # Default is Ollama OpenAI endpoint: http://localhost:11434/v1
        base_url = os.getenv("LOCAL_LLM_BASE_URL", os.getenv("LLAMA_BASE_URL", "http://localhost:11434/v1"))
        api_key = os.getenv("LOCAL_LLM_API_KEY", os.getenv("LLAMA_API_KEY", "ollama"))  # Ollama doesn't check tokens
        return ChatOpenAI(
            model=model_name,
            temperature=temperature,
            base_url=base_url,
            api_key=api_key,
        )
    else:
        raise ValueError(f"Unknown LLM provider: {provider}")


class StrictRAGAdvisor:
    """Strict RAG System: Returns 'No Answer' if there's no answer in documents or question is unrelated."""
    
    def __init__(self, vectorstore, llm_provider, llm_model, relevance_threshold, top_k, temperature, sigmoid_midpoint, sigmoid_steepness):
        self.vectorstore = vectorstore
        self.relevance_threshold = relevance_threshold
        self.top_k = top_k
        self.sigmoid_midpoint = sigmoid_midpoint
        self.sigmoid_steepness = sigmoid_steepness
        
        # 🔑 LLM is created only here → can swap later by just changing provider
        self.llm = create_llm(provider=llm_provider, model_name=llm_model, temperature=temperature)
        
        # Strict RAG prompt (not completely blocking, prioritizes documents but allows supplementary knowledge)
        rag_prompt_template = """You are an OLED Display Technical Assistant.
Answer the question using the provided context documents as your PRIMARY source.

RULES:
1. Always read the Context carefully and base your answer as much as possible on the Context.
2. If the Context contains partial but relevant information, you MAY use your own OLED/physics knowledge to fill in missing logical steps.
3. ONLY when the Context is clearly irrelevant or provides almost no signal, say: "Information not found in the provided OLED documents."
4. Never contradict the facts given in the Context.
5. Do NOT hallucinate specific numbers, experimental conditions, or paper titles that are not supported by the Context.

Context: {context}

Question: {question}

Answer:"""
        
        RAG_PROMPT = PromptTemplate(
            template = rag_prompt_template,
            input_variables = ["context", "question"]
        )
        
        self.rag_chain = RetrievalQA.from_chain_type(
            llm = self.llm,
            chain_type = "stuff",
            retriever = vectorstore.as_retriever(search_kwargs={"k": top_k}),
            chain_type_kwargs = {"prompt": RAG_PROMPT}
        )
        
    def get_relevance_score(self, query):
        # ChromaDB returns distance, so need to convert to similarity
        # Chroma L2 distance: lower is better (0 = identical)
        # Similarity = 1 - distance (assuming normalized embeddings)
        # But need to check what value langchain wrapper gives in similarity_search_with_score
        # LangChain Chroma's similarity_search_with_score returns L2 distance by default
        
        docs_with_scores = self.vectorstore.similarity_search_with_score(query, k=self.top_k)
        if not docs_with_scores:
            return 0.0
        
        # Score conversion (L2 distance -> Similarity)
        # If distance is 0, similarity is 1; if distance is 1+, similarity is low
        # Safely: similarity = 1.0 / (1.0 + distance)
        scores = []
        for doc, score in docs_with_scores:
             # Chroma returns distance by default. Lower is better.
             # Convert to 0-1 similarity score
             similarity = 1.0 / (1.0 + score)
             scores.append(similarity)
        
        avg_score = sum(scores) / len(scores)
        
        # Sigmoid transformation to spread score distribution
        sigmoid_score = 1 / (1 + math.exp(-self.sigmoid_steepness * (avg_score - self.sigmoid_midpoint)))
        return sigmoid_score
    
    def query(self, question):
        # Strict RAG Logic
        relevance_score = self.get_relevance_score(question)
        
        result = {
            "answer": None, 
            "mode": None, 
            "relevance_score": relevance_score,
            "retrieved_docs": []
        }
        
        # Check threshold (reject immediately if document similarity is low)
        if relevance_score >= self.relevance_threshold:
            result["mode"] = "RAG"
            result["retrieved_docs"] = self.vectorstore.similarity_search(question, k=self.top_k)
            # Execute RAG Chain (prompt instructs to say 'no information' if not found)
            rag_response = self.rag_chain.invoke({"query": question})["result"]
            result["answer"] = rag_response
            
            # Secondary check if LLM returned 'no information'
            if "Information not found" in rag_response or "provided context" in rag_response and "does not contain" in rag_response:
                 result["mode"] = "NO_ANSWER_IN_DOCS"
                 result["answer"] = "No Answer: The relevant content is not found in RAG documents."
        else:
            result["mode"] = "OFF_TOPIC"
            result["answer"] = "No Answer: The question is not related to OLED display or relevant documents are not available."
            logger.info(f"🚫 Low relevance rejection: {relevance_score:.3f}")
            
        return result

# Initialize Strict Advisor
advisor = StrictRAGAdvisor(
    vectorstore = vectorstore,
    llm_provider = LLM_PROVIDER,  # 🔑 Will be replaced with "llama_local" later
    llm_model = LLM_MODEL,
    relevance_threshold = RELEVANCE_THRESHOLD,
    top_k = TOP_K_DOCUMENTS,
    temperature = LLM_TEMPERATURE,
    sigmoid_midpoint = SIGMOID_MIDPOINT,
    sigmoid_steepness = SIGMOID_STEEPNESS
)

print("✅ Strict RAG Advisor (OLED) initialized successfully!")
print(f"📊 Strict Threshold = {RELEVANCE_THRESHOLD}")

## 📊 Enhanced Query Function with Monitoring

**What This Section Does:**
Wraps the advisor's `query` method with automatic monitoring, logging, and error handling.

**Why It's Needed:**
- Track performance metrics (response time, cost, token usage)
- Handle errors gracefully (API failures, timeouts)
- Log experiment data for hyperparameter tuning
- Systematically compare different configurations

**What Gets Tracked:**
- ⏱️ **Response Time**: Time each query takes (seconds)
- 💰 **Token Usage & Cost**: Input/output tokens and estimated API cost
- 📊 **Mode Distribution**: Number of queries per mode (RAG, NO_ANSWER, OFF_TOPIC, etc.)
- ⚠️ **Errors & Retries**: Automatic retry on transient failures (up to 2 attempts)

**How It Works:**
- The `monitored_query()` function wraps `advisor.query()`
- All metrics are automatically logged to the experiment tracker
- Returns the same structure as `advisor.query()` plus extra metadata
- The `ask()` helper provides a convenient one-line interface for quick queries

In [ ]:
# Cell 16: Enhanced Query Function with Monitoring
# This cell creates a wrapper function that adds monitoring, error handling, and cost tracking

def monitored_query(advisor, question, track_cost=True, log_to_experiment=True):
    """
    Enhanced query function with automatic monitoring and error handling.
    
    What This Function Does:
    1. Wraps advisor.query() with timing, error handling, and retry logic
    2. Tracks token usage and calculates estimated API cost
    3. Logs query results to experiment tracker for analysis
    4. Returns same format as advisor.query() + additional metadata
    
    Args:
        advisor: StrictRAGAdvisor instance (the main advisor object for OLED RAG)
        question: User's question text
        track_cost: Whether to track token usage and cost (default: True)
    
    Returns:
        dict: Result dictionary with answer, metadata, response_time, token counts
    """
    start_time = time.time()  # Start timer for response time calculation
    result = None
    error_occurred = False
    
    try:
        logger.info(f"🔍 Processing query: {question[:50]}...")
        
        # Wrap advisor.query() with automatic retry on API errors
        # @retry_on_api_error automatically retries up to 2 times on failures
        @retry_on_api_error(max_retries=2, delay=2)
        def query_with_retry():
            return advisor.query(question)
        
        # Call the wrapped query method
        result = query_with_retry()
        
        # Calculate response time (elapsed time since start)
        response_time = time.time() - start_time
        result['response_time'] = response_time
        
        # Track token usage and cost if enabled
        if track_cost:
            # Estimate token usage based on input/output text length
            # cost_tracker uses tiktoken to count tokens accurately
            input_text = question
            output_text = result['answer']
            input_tokens, output_tokens = cost_tracker.add_tokens(input_text, output_text)
            
            # Store token counts in result for display/analysis
            result['input_tokens'] = input_tokens
            result['output_tokens'] = output_tokens
        
        # Log successful query completion with key metrics
        mode = result['mode']
        logger.info(f"✅ Query completed in {response_time:.2f}s | Mode: {mode} | Relevance: {result['relevance_score']:.3f}")
        
        # Track this query in experiment tracker for statistical analysis
        # This accumulates data for comparing different hyperparameter settings
        if log_to_experiment and hasattr(experiment_tracker, 'current_experiment') and experiment_tracker.current_experiment:
            experiment_tracker.log_query(response_time, mode)
        
        return result
        
    except Exception as e:
        # Error handling: Catch any exceptions and return error result
        error_occurred = True
        response_time = time.time() - start_time
        
        logger.error(f"❌ Query failed after {response_time:.2f}s: {type(e).__name__} - {str(e)}")
        
        # Return error result with user-friendly message
        return {
            'mode': 'ERROR',
            'answer': f"Sorry, an error occurred while processing your question: {str(e)}",
            'relevance_score': 0.0,
            'response_time': response_time,
            'error': str(e)
        }

# Create a convenience function that uses the global advisor
def ask(question):
    """
    Convenient shortcut function to ask a question with automatic monitoring.
    
    This is a simpler interface - just call ask(question) instead of 
    monitored_query(advisor, question).
    
    Usage:
        result = ask("What are the key degradation mechanisms of a blue phosphorescent OLED?")
        print(result['answer'])
    
    Args:
        question: User's question text
    
    Returns:
        dict: Result dictionary with answer and metadata
    """
    return monitored_query(advisor, question)

print("✅ Monitored query function ready!")
print("💡 Use ask(question) for quick queries with automatic monitoring")

## 🧪 Testing the Strict RAG System

Test the advisor with sample questions to see how RAG, NO_ANSWER, and OFF_TOPIC modes are selected.

---
### 📊 Quick Reference: The Score System

```
┌─────────────────────────────────────────────────────────────────┐
│  RELEVANCE SCORE (0.0 - 1.0)                                    │
│  Question: "Do the documents match the query?"                  │
│  • 0.85 → Documents are highly relevant to the query ✓          │
│  • 0.54 → Documents somewhat match (uncertain zone)             │
│  • 0.30 → Documents are not relevant to the query ✗             │
└─────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────┐
│  RAG SCORE (1-10)                                               │
│  Question: "How good is the RAG answer?"                        │
│  • Measures: Specificity + Relevance + Factuality               │
│  • 8.5 → High-quality answer with concrete facts from docs      │
│  • 6.0 → Decent answer but lacks detail                         │
└─────────────────────────────────────────────────────────────────┘

💡 KEY INSIGHTS:
   1. Low Relevance = No OLED-related documents → OFF_TOPIC rejection
   2. High Relevance + RAG "Information not found" = Relevant documents exist but no specific answer → NO_ANSWER
   3. High Relevance + Successful answer = Accurate document-based answer → RAG mode
```

In [ ]:
# Cell 18: Execute Advisor Core Logic with Test Questions
# Execute advisor with predefined test questions and format the results for output

# HTML formatting function - for better-looking result output
def show_response(result, question):
    """
    Display advisor response in a formatted HTML box.
    Applied for Strict RAG mode (Project 2).
    """
    mode = result['mode']
    answer = result['answer']
    relevance_score = result['relevance_score']
    
    # Color coding by mode (Strict RAG: Only use RAG, NO_ANSWER_IN_DOCS, OFF_TOPIC)
    if mode == "RAG":
        color = "#4caf50"  # Green
        icon = "🟢"
        mode_text = "RAG Mode (Document-based Response)"
    elif mode == "NO_ANSWER_IN_DOCS":
        color = "#ff9800"  # Orange
        icon = "🟠"
        mode_text = "No Answer (No Relevant Content in Documents)"
    elif mode == "OFF_TOPIC":
        color = "#f44336"  # Red
        icon = "🔴"
        mode_text = "Off-Topic Rejection (Low Relevance, Auto-Rejected)"
    elif mode == "ERROR":
        color = "#f44336"  # Red
        icon = "❌"
        mode_text = "System Error"
    else:
        color = "#9e9e9e"  # Gray
        icon = "⚪"
        mode_text = f"Unknown mode: {mode}"
    
    # Generate metadata line (includes relevance score)
    metadata = f"{icon} <strong>{mode_text}</strong> | Relevance Score: {relevance_score:.3f}"
    
    html = f"""
    <div style="
        background-color:#f9f9f9;
        border-left: 6px solid {color};
        padding: 15px;
        margin: 15px 0;
        border-radius: 4px;
        font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Arial, sans-serif;">
        <div style="color:#333; font-size:14px; margin-bottom:10px;">
            <strong>Question:</strong> {question}
        </div>
        <div style="color:#666; font-size:13px; margin-bottom:10px;">
            {metadata}
        </div>
        <div style="color:#111; font-size:14px; line-height:1.6; white-space: pre-wrap;">
            <strong>Answer:</strong><br>{answer}
        </div>
    </div>
    """
    display(HTML(html))

# Execute advisor core logic with test questions
print("🧪 Executing advisor core logic with test questions...\n")

# Start experiment tracking
experiment_tracker.start_experiment({
    # RAG Quality (Step 1)
    'chunk_size': CHUNK_SIZE,
    'chunk_overlap': CHUNK_OVERLAP,
    'top_k': TOP_K_DOCUMENTS,
    'temperature': LLM_TEMPERATURE,
    
    # System Architecture (Step 2) - Only what's used in Strict RAG
    'relevance_threshold': RELEVANCE_THRESHOLD,
    'sigmoid_midpoint': SIGMOID_MIDPOINT,
    'sigmoid_steepness': SIGMOID_STEEPNESS
})

# Reset cost tracker for this experiment
cost_tracker.reset()

# Store relevance scores for later average calculation
relevance_scores = []

for i, question in enumerate(TEST_QUERIES, 1):
    print(f"\n{'='*80}")
    print(f"Test {i}/{len(TEST_QUERIES)}")
    print(f"{'='*80}")
    
    # Use monitored_query() instead of advisor.query()
    result = monitored_query(advisor, question)
    show_response(result, question)
    
    # Save relevance score
    if 'relevance_score' in result:
        relevance_scores.append(result['relevance_score'])
    
    # Display performance metrics
    if 'response_time' in result:
        print(f"⏱️ Response time: {result['response_time']:.2f}s")
    if 'input_tokens' in result and 'output_tokens' in result:
        print(f"🎫 Tokens: {result['input_tokens']} input + {result['output_tokens']} output = {result['input_tokens'] + result['output_tokens']} total")

# Print experiment summary
print(f"\n{'='*80}")
print("📊 Experiment Summary")
print(f"{'='*80}")
cost_summary = cost_tracker.get_summary()
print(f"💰 Total tokens: {cost_summary['total_tokens']:,}")
print(f"💵 Estimated cost: ${cost_summary['estimated_cost_usd']:.4f}")
print(f"📊 Mode distribution:")
for mode, count in experiment_tracker.current_experiment['mode_counts'].items():
    if count > 0:
        print(f"   - {mode}: {count}")

## 📊 Quantitative Evaluation: System Quality

**What This Section Does:**
Evaluates the quality of answers generated by the Strict RAG system.

**Evaluation Process:**
1. **For each test question:**
   - Generate answer using Strict RAG Advisor
   - If answer is provided (not rejected), score it using LLM-as-a-judge

2. **Evaluation Metrics** (1-10 points):
   - **Specificity**: How detailed is the answer?
   - **Relevance**: Does it directly answer the question?
   - **Factuality**: Does it contain verifiable facts from the documents?

In [ ]:
# Cell 20: Quantitative Evaluation - System Quality Check
# This cell implements the RAGEvaluator class for evaluating system performance.
# Note: LLM baseline comparison has been removed to focus on Strict RAG quality.

import json
import pandas as pd

class RAGEvaluator:
    """Evaluate the final advisor answer quality."""
    
    def __init__(self, llm, advisor):
        self.llm = llm
        self.advisor = advisor
    
    def get_final_system_answer(self, question):
        """Get the final answer from the full advisor system."""
        # track_cost=False, log_to_experiment=False to avoid double counting
        result = monitored_query(self.advisor, question, track_cost=False, log_to_experiment=False)
        return result
    
    def score_answer(self, question, answer):
        """Score an answer on specificity, relevance, and factuality (1-10)."""
        evaluation_prompt = f"""You are an expert evaluator. Score the following answer on these three criteria (scale 1-10):

1. SPECIFICITY: How specific and detailed is the answer? (1 = vague, 10 = very specific with details)
2. RELEVANCE: How relevant is the answer to the question? (1 = off-topic, 10 = directly answers question)
3. FACTUALITY: Does the answer contain verifiable facts and data? (1 = no facts/opinions only, 10 = rich with facts and data)

Question: {question}

Answer: {answer}

Respond ONLY with a JSON object in this exact format (no other text):
{{"specificity": <score>, "relevance": <score>, "factuality": <score>}}"""
        
        try:
            response = self.llm.invoke(evaluation_prompt).content
            start_idx = response.find('{')
            end_idx = response.rfind('}') + 1
            json_str = response[start_idx:end_idx]
            scores = json.loads(json_str)
            return scores
        except:
            return {"specificity": 5, "relevance": 5, "factuality": 5}
    
    def evaluate_system(self, question):
        """Evaluate final system answer quality."""
        result = self.get_final_system_answer(question)
        final_answer = result.get("answer", "")
        mode = result.get("mode", "UNKNOWN")
        
        # If no answer (rejected), we can't score specificity/factuality nicely
        # But we can track that it was rejected.
        if mode in ["OFF_TOPIC", "NO_ANSWER_IN_DOCS"]:
             return {
                "question": question,
                "final_answer": final_answer,
                "mode": mode,
                "final_specificity": 0,
                "final_relevance": 0,
                "final_factuality": 0,
                "final_word_count": 0
            }
            
        # Score the answer
        final_scores = self.score_answer(question, final_answer)
        final_word_count = len(final_answer.split())
        
        return {
            "question": question,
            "final_answer": final_answer,
            "mode": mode,
            "final_specificity": final_scores["specificity"],
            "final_relevance": final_scores["relevance"],
            "final_factuality": final_scores["factuality"],
            "final_word_count": final_word_count,
        }

# Initialize evaluator
evaluator = RAGEvaluator(advisor.llm, advisor)

# Run evaluation
print("📊 Running system quality evaluation...\n")
evaluation_results = []

for i, question in enumerate(TEST_QUERIES, 1):
    print(f"Evaluating {i}/{len(TEST_QUERIES)}: {question[:50]}...")
    result = evaluator.evaluate_system(question)
    evaluation_results.append(result)
    print("  ✓ Complete")

print("\n✅ Evaluation complete!")

## 📈 Evaluation Results Analysis
This section analyzes the performance of the Strict RAG system, calculating quality scores for answered questions and tracking rejection counts.

In [ ]:
# Cell 22: System Evaluation Analysis
# This cell displays performance metrics for the Strict RAG system.

# Convert results to DataFrame
df = pd.DataFrame(evaluation_results)

# Filter only answered questions for scoring stats (exclude rejections)
answered_df = df[~df['mode'].isin(['OFF_TOPIC', 'NO_ANSWER_IN_DOCS'])]

if not answered_df.empty:
    avg_specificity = answered_df['final_specificity'].mean()
    avg_relevance = answered_df['final_relevance'].mean()
    avg_factuality = answered_df['final_factuality'].mean()
    avg_words = answered_df['final_word_count'].mean()
    
    # Calculate overall score
    answered_df['final_overall'] = (answered_df['final_specificity'] + answered_df['final_relevance'] + answered_df['final_factuality']) / 3
    avg_overall = answered_df['final_overall'].mean()
else:
    avg_specificity = 0
    avg_relevance = 0
    avg_factuality = 0
    avg_words = 0
    avg_overall = 0

# Display summary statistics
print("=" * 80)
print("📊 STRICT RAG SYSTEM PERFORMANCE SUMMARY")
print("=" * 80)
print(f"Total Questions: {len(df)}")
print(f"Answered: {len(answered_df)}")
print(f"Rejected/No Data: {len(df) - len(answered_df)}")
print("-" * 80)
print(f"{'Metric (Answered Queries Only)':<35} {'Score':<15}")
print("-" * 80)
print(f"{'Specificity (1-10)':<35} {avg_specificity:<15.2f}")
print(f"{'Relevance (1-10)':<35} {avg_relevance:<15.2f}")
print(f"{'Factuality (1-10)':<35} {avg_factuality:<15.2f}")
print(f"{'Overall Score (1-10)':<35} {avg_overall:<15.2f}")
print(f"{'Average Word Count':<35} {avg_words:<15.1f}")
print()

# Display detailed results table
print("=" * 80)
print("📋 DETAILED RESULTS BY QUESTION")
print("=" * 80)
display(df[['question', 'mode', 'final_answer']])

# Calculate average relevance score from Cell 18's data
# relevance_scores list is collected from Cell 18
if 'relevance_scores' in dir() and len(relevance_scores) > 0:
    avg_relevance_score = sum(relevance_scores) / len(relevance_scores)
else:
    avg_relevance_score = None
    print("⚠️ Warning: relevance_scores not found. Run Cell 18 first.")

# Calculate Q1–Q2 vs Q5–Q6 separation (for CSV recording)
relevance_mean_pos = None
relevance_mean_neg = None
relevance_gap = None
relevance_margin = None

try:
    if 'relevance_scores' in dir() and len(relevance_scores) >= 6:
        # Q1, Q2 → OLED core questions (Positive)
        pos_indices = [0, 1]
        # Q5, Q6 → Completely OFF-TOPIC questions (Negative)
        neg_indices = [4, 5]

        pos_scores = [float(relevance_scores[i]) for i in pos_indices]
        neg_scores = [float(relevance_scores[i]) for i in neg_indices]

        relevance_mean_pos = sum(pos_scores) / len(pos_scores)
        relevance_mean_neg = sum(neg_scores) / len(neg_scores)
        relevance_gap = relevance_mean_pos - relevance_mean_neg
        relevance_margin = min(pos_scores) - max(neg_scores)
    else:
        print("⚠️ Need at least 6 relevance scores (Q1–Q6) to compute separation metrics.")
except Exception as e:
    print("⚠️ Failed to compute relevance separation metrics:", e)

# Save experiment results
if hasattr(experiment_tracker, 'save_experiment'):
    # notes: Store simple summary text (also record which LLM configuration)
    notes_str = (
        f"model={LLM_MODEL}, provider={LLM_PROVIDER}, "
        f"Strict RAG Evaluation: {len(answered_df)}/{len(df)} answered."
    )

    experiment_tracker.save_experiment(
        avg_relevance_score=avg_relevance_score,
        avg_rag_score=avg_overall,
        avg_rag_specificity=avg_specificity,
        avg_rag_relevance=avg_relevance,
        avg_rag_factuality=avg_factuality,
        rel_mean_pos=relevance_mean_pos,
        rel_mean_neg=relevance_mean_neg,
        rel_gap=relevance_gap,
        rel_margin=relevance_margin,
        notes=notes_str,
    )

    # Console output always displays in the same format
    mean_pos_str = "N/A" if relevance_mean_pos is None else f"{relevance_mean_pos:.3f}"
    mean_neg_str = "N/A" if relevance_mean_neg is None else f"{relevance_mean_neg:.3f}"
    gap_str = "N/A" if relevance_gap is None else f"{relevance_gap:.3f}"
    margin_str = "N/A" if relevance_margin is None else f"{relevance_margin:.3f}"

    print(f"\n📊 Average Relevance Score: {avg_relevance_score:.3f}" if avg_relevance_score else "")
    print(
        f"📊 Relevance separation (Q1–Q2 vs Q5–Q6): "
        f"mean_pos={mean_pos_str}, "
        f"mean_neg={mean_neg_str}, "
        f"gap={gap_str}, margin={margin_str}"
    )
    print("✅ Experiment results saved to CSV.")

## 💬 Interactive Q&A Interface
Now you can ask the AI-Driven OLED Assistant your own questions directly!

In [ ]:
# Cell 26: Interactive Q&A Interface
# ⚠️ This cell is handled so handlers don't get registered multiple times even if run multiple times

import ipywidgets as widgets
from IPython.display import clear_output, display, HTML

# Global flag to prevent duplicate registration
if 'OLED_UI_INITIALIZED' not in dir():
    OLED_UI_INITIALIZED = False

# Remove existing UI if present
if OLED_UI_INITIALIZED:
    try:
        oled_ui_box.close()
    except:
        pass

# Create widgets
oled_question_input = widgets.Textarea(
    value='',
    placeholder='Enter your OLED Display related question...',
    description='Question:',
    layout=widgets.Layout(width='90%', height='80px'),
    style={'description_width': '50px'}
)

oled_submit_btn = widgets.Button(
    description='🔍 Get Answer',
    button_style='primary',
    layout=widgets.Layout(width='150px')
)

oled_output = widgets.Output()

def oled_handle_submit(btn):
    with oled_output:
        clear_output(wait=True)
        question = oled_question_input.value.strip()
        if not question:
            print("⚠️ Please enter a question!")
            return
        
        print("🔍 Analyzing question...")
        result = advisor.query(question)
        
        mode = result['mode']
        answer = result['answer']
        score = result['relevance_score']
        
        if mode == "RAG":
            color, icon, text = "#4caf50", "🟢", "RAG Mode (Document-based)"
        elif mode == "NO_ANSWER_IN_DOCS":
            color, icon, text = "#ff9800", "🟠", "No Answer (Not in Documents)"
        else:
            color, icon, text = "#f44336", "🔴", "Off-Topic (Rejected)"
        
        html = f'''<div style="background:#f5f5f5; border-left:5px solid {color}; padding:12px; margin:8px 0; border-radius:4px;">
<b>Q:</b> {question}<br>
<span style="color:#666">{icon} {text} | Relevance: {score:.3f}</span><br><br>
<b>A:</b> {answer}
</div>'''
        display(HTML(html))
        
        if mode == 'RAG' and result.get('retrieved_docs'):
            print("\n📄 Reference Documents:")
            for i, doc in enumerate(result['retrieved_docs'][:2], 1):
                print(f"  {i}. {doc.page_content[:120]}...")

oled_submit_btn.on_click(oled_handle_submit)

# Wrap in UI box
oled_ui_box = widgets.VBox([
    widgets.HTML("<h3>💬 AI-Driven OLED Assistant</h3>"),
    oled_question_input,
    oled_submit_btn,
    oled_output
])

OLED_UI_INITIALIZED = True
display(oled_ui_box)